In [14]:
import pandas as pd
import sys
import os

# Add the src directory to the path so we can import our modules
sys.path.append(os.path.abspath('..'))

In [15]:
# import os
# os.getcwd()
# # set to 1 level up
# os.chdir(os.path.dirname(os.getcwd()))
# os.getcwd()


In [16]:
merchants = pd.read_csv("data/merchants.csv")
merchants=merchants.rename(columns={'country': 'merchant_country'})
merchants.head()

,merchant_id,category,merchant_country,trust_score,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history
0,M0001,travel,Austria,1.000000,3,97.23,84,0
1,M0002,clothing,Poland,0.689695,2,142.71,93,1
2,M0003,electronics,Czech Republic,0.834610,5,67.83,65,0
3,M0004,electronics,Belgium,0.607229,1,16.77,9,1
4,M0005,grocery,Ireland,0.641901,2,38.56,100,0


In [17]:
users = pd.read_csv("data/users.csv")
users=users.rename(columns={'country': 'user_country'})
users.head()

,user_id,age,sex,education,primary_source_of_income,sum_of_monthly_installments,sum_of_monthly_expenses,user_country,signup_date,risk_score
0,U00001,56,Other,High School,Employment,477.69,243.18,Finland,2021-04-01,0.571079
1,U00002,36,Female,Bachelor,Business,31.60,737.76,France,2020-07-07,0.705268
2,U00003,70,Other,High School,Student Aid,275.72,1477.50,Finland,2022-08-23,0.429579
3,U00004,38,Other,High School,Savings,288.29,1135.03,Belgium,2022-05-21,0.581384
4,U00005,61,Male,High School,Employment,7.00,892.76,Italy,2022-03-15,0.420227


In [18]:
transactions = pd.read_json("data/transactions.json", lines=True)
transactions.head()

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,location,payment_method,is_international,session_length_seconds,is_first_time_merchant,is_fraud
0,TX000000,2022-06-17 23:28:00,U14804,M0314,130.03,in-store,EUR,Android,"{'lat': 40.057938, 'long': 14.959737}",debit_card,1,145,0,0
1,TX000001,2022-01-04 15:39:00,U16634,M0675,132.00,online,EUR,Android,"{'lat': 42.05935, 'long': 1.596625}",debit_card,1,32,1,0
2,TX000002,2022-09-09 21:58:00,U18005,M0479,8.65,online,EUR,Android,"{'lat': 50.923432, 'long': 21.795262}",credit_card,1,604,1,0
3,TX000003,2023-11-20 06:40:00,U13690,M0538,19.82,mobile,EUR,iOS,"{'lat': 56.833396, 'long': -9.881566}",credit_card,0,1031,1,0
4,TX000004,2022-04-28 08:08:00,U04642,M0128,101.92,in-store,EUR,Android,"{'lat': 54.042996, 'long': 2.168516}",credit_card,1,330,0,0


In [19]:
trans_user = pd.merge(transactions, users, on='user_id', how='left')

In [20]:
df_merged = pd.merge(trans_user, merchants, on='merchant_id', how='left')

## Feature Engineering

Adding time-based features for transaction analysis

In [21]:
# Import the feature engineering function
from src.feature_engineering.time_features import calculate_time_since_last_transaction

# Apply the function to calculate time since last transaction
df_merged = calculate_time_since_last_transaction(df_merged)

In [22]:
df_merged

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,location,payment_method,...,signup_date,risk_score,category,merchant_country,trust_score,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history,time_since_last_txn
290822,TX290822,2022-01-08 10:03:00,U00001,M0906,19.09,online,EUR,Web,"{'lat': 69.291741, 'long': -7.72625}",debit_card,...,2021-04-01,0.571079,grocery,Belgium,0.631421,2,26.85,37,0,-9.0
197951,TX197951,2022-01-08 16:48:00,U00001,M0245,32.24,online,EUR,Android,"{'lat': 54.710715, 'long': 15.368695}",mobile_payment,...,2021-04-01,0.571079,gaming,Spain,0.859810,2,197.43,115,1,24300.0
187979,TX187979,2022-01-12 16:23:00,U00001,M0332,4.29,in-store,EUR,Android,"{'lat': 58.897717, 'long': 30.25543}",debit_card,...,2021-04-01,0.571079,grocery,Spain,0.934815,5,18.36,68,0,344100.0
137034,TX137034,2022-01-18 10:40:00,U00001,M0331,67.00,in-store,EUR,iOS,"{'lat': 42.684202, 'long': 3.843828}",debit_card,...,2021-04-01,0.571079,gaming,Belgium,0.859404,6,123.91,94,1,497820.0
401318,TX401318,2022-07-18 08:05:00,U00001,M0971,3.59,in-store,EUR,Web,"{'lat': 44.296089, 'long': 15.749937}",mobile_payment,...,2021-04-01,0.571079,clothing,Germany,0.516453,1,0.98,52,1,15629100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263172,TX263172,2023-08-16 08:18:00,U20000,M0426,35.83,in-store,EUR,iOS,"{'lat': 58.083086, 'long': 36.673547}",debit_card,...,2022-08-26,0.607373,restaurants,Netherlands,0.915723,7,158.60,15,1,4356000.0
360206,TX360206,2023-08-28 07:39:00,U20000,M0487,21.69,mobile,EUR,Web,"{'lat': 50.804658, 'long': 16.073549}",bank_transfer,...,2022-08-26,0.607373,travel,Italy,1.000000,3,25.54,14,1,1034460.0
358294,TX358294,2023-09-15 07:45:00,U20000,M0468,39.16,in-store,EUR,iOS,"{'lat': 37.929143, 'long': -3.931207}",mobile_payment,...,2022-08-26,0.607373,clothing,Finland,0.413113,4,86.73,25,0,1555560.0
251399,TX251399,2023-11-11 18:24:00,U20000,M0830,134.76,in-store,EUR,Android,"{'lat': 64.194035, 'long': -8.155081}",mobile_payment,...,2022-08-26,0.607373,restaurants,Germany,0.531957,2,45.86,87,0,4963140.0
